In [1]:
import os, sys
sys.path.append(os.path.abspath(".."))

# import requests
# import scrapy
# import time
# import pandas as pd
# from pprint import pprint
# import re
# import json
# import datetime

from scrapers import staff

In [2]:
data = staff.main()

Mon Sep 20 19:33:55 2021
Starting crawling 'https://staff.am'.
1412 jobs detected.
Starting crawling job postings..


  0%|          | 3/1412 [00:05<45:39,  1.94s/it]


            New 'h3' fields in job description list are present:
            	Check the output.
            


  0%|          | 4/1412 [00:07<45:07,  1.92s/it]


            New 'h3' fields in job description list are present:
            	Check the output.
            


...


            New 'h3' fields in job description list are present:
            	Check the output.
            


100%|██████████| 1412/1412 [46:36<00:00,  1.98s/it]


1412/1412 jobs scraped.
Saving files.
Company_Title: 1412
Company_URL: 1412
URL: 1412
Total_views: 1412
Followers: 1412
Active_Jobs: 1412
Jobs_History: 1412
Job_Views: 1412
Job_Title: 1412
Application_Deadline: 1412
Industry: 1412
Employment_term: 1412
Job_Category: 1412
Job_type: 1412
Job_Location: 1412
Salary: 1412
Job_description: 1412
Job_responsibilities: 1412
Required_qualifications: 1412
Required_candidate_level: 1412
Additional_information: 1412
Professional_skills: 1412
Soft_skills: 1412
: 18
Are_you_our_future_Sparker_colleague_?_Apply_now_or_send_me_your_CV_at: 2
https//www.sparkers.com/: 3
Big_data_developer_Spark_(Scala_/_Python): 1
Anyone_interested_in_the_job_can_send_their_CV_to: 1
Interested_candidates_should_submit_a_resume_via_our_career_portal_ by_clicking: 13
Personal_Qualities: 1
Interested_candidates_should_submit_a_resume_and_portfolio_via_our_career_portal_ by_clicking: 9
Interested_candidates_should_submit_a_resume via_our_career_portal_ by_clicking: 9
About_t

100%|██████████| 1/1 [00:00<00:00,  1.44it/s]

Extracted 1/1
Mon Sep 20 20:21:00 2021
Ended crawling 'https://staff.am'.


In [ ]:
a.iterable